# Weatherdata for IPM Plateform

## 1. import weather data modules

In [1]:
from weatherdata.ipm import WeatherDataHub 

## 2. Access to weatherdatasource available on the platefom

In [2]:
ws=WeatherDataHub()

In [ ]:
ws.list_resources()

## 3. Get ressource for a specific weatherdataresource 

To connect to the meteo resource, simply enter the name of the meteo resource in the get_ressource function. 

In [3]:
fmi=ws.get_ressource(name='Finnish Meteorological Institute measured data')

Once connected to the resource various functions are available. You can check 
* the available stations for the resource with station_ids function. This function This function returns a dataframe of the stations available for the resource with their names, their identifiers and the coordinates of the station (latitude, longitude)
* the weather parameters accepted by the resource  with parameters function. This function return common and optional weather parameter 
* Get Weatherdata in xarray dataset with attribute or json format (more description of this function below)

In [ ]:
fmi.station_ids()

In [ ]:
fmi.parameters()

### Get weather data

According to weather resources differents parameters can be used:
* *for historic weatherdata*
    * parameters: list of weather parameters available for the resource selected
    * station_id: list of station id available for the resource selected
    * timeStart: Start date of the request
    * timeEnd: End date of the request
    * timezone: The timezone
    * format: 'ds' to obtain a xarray dataset or 'json' to obtain the json     
    
    
* *for forecasts weather resources*
    * latitude: list of latitude
    * longitude: list of longitude
    * altitude: list of altitude

#### Example for historic weather ressources 
* for one station_id

In [ ]:
%pdb

In [ ]:
# for one station_ids
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds',
            varname='id')
ds

In [ ]:
ds.to_dataframe()

In [ ]:
temp = ds['1002']
temp.plot()

* for several station_ids

In [ ]:
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533,101185],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds',varname='name')
ds

In [ ]:
ds.to_dataframe().dropna()

In [ ]:
ds.plot.scatter(x='time',y='Mean air temperature at 2m',hue='location')
temp = ds['Mean air temperature at 2m'].isel(time=0)

In [ ]:
temp.isel(location=0).plot

#### For forecast weather resources
* example for one latitude, longitude, altitude

In [ ]:
norway=ws.get_ressource(name='Met Norway Locationforecast')

In [ ]:
ds=norway.data(latitude=[67.2828], longitude=[14.3711], altitude=[70],format='ds')
ds

In [ ]:
ds.to_dataframe()

In [ ]:
ds=norway.data(latitude=[67.2828,61.27], longitude=[14.3711,25.52], altitude=[70, 0],format='ds')
ds

**From ds you can see data as dataframe with function to_dataframe and exclude na value with dropna**
for more information on the http://xarray.pydata.org/en/stable/index.html

In [ ]:
ds.to_dataframe().dropna()

#### Example of weatherdatasource with credentials

In [ ]:
fruitdevis=ws.get_ressource(name='Fruitweb')
fruitdevis.data(parameters=[1002,3002],stationId=[536], timeStart='2021-02-01',timeEnd='2021-03-01',credentials={"userName":"","password":"GF90esoleo"})

# Metpy 

In [ ]:
import metpy.calc as mpcalc
from metpy.units import units
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds')
ds

In [ ]:
temp = ds['1002']*units.degC
hum=ds['3002']*units.percent

In [ ]:
mpcalc.dewpoint_from_relative_humidity(temp, hum)